In [4]:
import requests
import json
import time
import subprocess
import os
import random
from tqdm import tqdm

In [5]:
def save_last_id(last_id):
    """حفظ آخر معرف تمت معالجته"""
    with open('last_processed_id.json', 'w', encoding='utf-8') as f:
        json.dump({'last_id': last_id}, f, ensure_ascii=False, indent=4)

def load_last_id():
    """تحميل آخر معرف تمت معالجته"""
    try:
        with open('last_processed_id.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data.get('last_id', 865)  # نبدأ من 865 إذا لم يكن هناك ملف
    except FileNotFoundError:
        return 865  # نبدأ من 865 إذا لم يكن هناك ملف

def load_existing_data():
    """تحميل البيانات الموجودة مسبقاً"""
    try:
        with open('all_hadiths_explanations2.json', 'r', encoding='utf-8') as f:
            return json.load(f)
    except FileNotFoundError:
        return []

def save_data(all_explanations, successful_ids):
    """حفظ البيانات في الملفات"""
    # حفظ جميع الشروحات في ملف JSON
    with open('all_hadiths_explanations2.json', 'w', encoding='utf-8') as f:
        json.dump(all_explanations, f, ensure_ascii=False, indent=4)
    
    # حفظ معرفات الأحاديث التي تم العثور على شروحها 
    with open('successful_hadith_ids2.json', 'w', encoding='utf-8') as f:
        json.dump(successful_ids, f, ensure_ascii=False, indent=4)

In [6]:
def restart_server():
    """إعادة تشغيل السيرفر بدون طباعة رسائل كثيرة"""
    try:
        # إيقاف جميع عمليات Node.js بطريقة آمنة
        try:
            if os.name == 'nt':  # Windows
                subprocess.run(["taskkill", "/F", "/IM", "node.exe"], 
                               shell=True, capture_output=True, timeout=30)
            else:  # Linux/Mac
                subprocess.run(["pkill", "-f", "node"], 
                               shell=True, capture_output=True, timeout=30)
        except:
            pass
        
        # انتظار للتأكد من إغلاق جميع العمليات
        time.sleep(3)
        
        # تغيير المسار إلى مجلد السيرفر
        server_dir = "D:/repo/Markting/dorar-hadith-api"
        original_dir = os.getcwd()  # حفظ المسار الحالي للعودة إليه لاحقًا
        
        if not os.path.exists(server_dir):
            raise Exception(f"مجلد السيرفر غير موجود: {server_dir}")
        
        os.chdir(server_dir)
        
        # تشغيل السيرفر في الخلفية بدون نافذة عائمة
        if os.name == 'nt':  # Windows
            # استخدام subprocess.CREATE_NO_WINDOW لتشغيل بدون نوافذ عائمة
            startupinfo = subprocess.STARTUPINFO()
            startupinfo.dwFlags |= subprocess.STARTF_USESHOWWINDOW
            startupinfo.wShowWindow = 0  # SW_HIDE
            
            # استخدام CMD مع /c لإخفاء النافذة
            process = subprocess.Popen(
                "cmd /c npm start",
                shell=True,
                startupinfo=startupinfo,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
        else:  # Linux/Mac
            process = subprocess.Popen(
                "npm start > server.log 2>&1 &",
                shell=True
            )
        
        # العودة إلى المسار الأصلي
        os.chdir(original_dir)
        
        # انتظار حتى يبدأ السيرفر مع محاولات متعددة
        max_retries = 12
        retry_count = 0
        server_started = False
        
        while retry_count < max_retries and not server_started:
            try:
                time.sleep(5)  
                response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
                if response.status_code == 200:
                    server_started = True
                    break
            except requests.RequestException:
                retry_count += 1
        
        if not server_started:
            # محاولة بديلة لبدء السيرفر
            try:
                os.chdir(server_dir)
                
                if os.name == 'nt':  # Windows
                    startupinfo = subprocess.STARTUPINFO()
                    startupinfo.dwFlags |= subprocess.STARTF_USESHOWWINDOW
                    startupinfo.wShowWindow = 0  # SW_HIDE
                    
                    subprocess.Popen(
                        "cmd /c npm start",
                        shell=True,
                        startupinfo=startupinfo,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE
                    )
                else:  # Linux/Mac
                    os.system("npm start > server.log 2>&1 &")
                
                os.chdir(original_dir)
                
                time.sleep(10)
                
                response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
                if response.status_code == 200:
                    server_started = True
            except:
                pass
                
        if not server_started:
            raise Exception("فشل في بدء تشغيل السيرفر بعد كل المحاولات")
            
    except Exception as e:
        raise

In [7]:
def test_server_health():
    """اختبار حالة السيرفر"""
    try:
        response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
        return response.status_code == 200
    except:
        return False

def crawl_hadith_explanations():
    base_url = "http://localhost:5000/v1/site/sharh/"
    all_explanations = load_existing_data()
    start_id = load_last_id() + 1  # نبدأ من ID التالي
    max_id = 88430
    batch_size = 70
    
    # تحميل الـ IDs الناجحة السابقة إذا كانت موجودة
    try:
        with open('successful_hadith_ids2.json', 'r', encoding='utf-8') as f:
            successful_ids = json.load(f)
    except FileNotFoundError:
        successful_ids = []
    
    # حذف التكرارات من قائمة الـ IDs الناجحة
    successful_ids = list(set(successful_ids))
    
    # استخراج الـ IDs الموجودة في البيانات المحفوظة
    existing_ids = [item['hadith_id'] for item in all_explanations]
    
    print(f"بدء العملية من معرف الحديث: {start_id}")
    print(f"عدد الشروحات المحفوظة مسبقًا: {len(all_explanations)}")
    print(f"عدد الأحاديث الناجحة المسجلة: {len(successful_ids)}")
    
    # التأكد من أن السيرفر يعمل قبل البدء
    if not test_server_health():
        print("❌ السيرفر غير متاح، جاري إعادة تشغيله قبل البدء...")
        restart_server()
    
    current_batch_count = 0
    found_count = 0
    skipped_count = 0
    failed_count = 0
    
    # إنشاء شريط التقدم باستخدام tqdm مع عرض إحصائيات محدثة باستمرار
    total_ids = max_id - start_id + 1
    
    # تحديث البار الرئيسي
    def update_progress_bar(pbar, found, skipped, failed, current_id):
        pbar.set_postfix({
            "عثر": found,
            "تخطى": skipped,
            "فشل": failed,
            "آخر_معالجة": current_id
        })
    
    with tqdm(total=total_ids, desc="معالجة الأحاديث", unit="حديث") as pbar:
        for hadith_id in range(start_id, max_id + 1):
            try:
                # تخطي الأحاديث التي تمت معالجتها مسبقًا
                if hadith_id in existing_ids or hadith_id in successful_ids:
                    skipped_count += 1
                    update_progress_bar(pbar, found_count, skipped_count, failed_count, hadith_id)
                    pbar.update(1)
                    continue
                
                # فحص حالة السيرفر بشكل عشوائي
                if random.random() < 0.05:  # 5% احتمالية للاختبار
                    if not test_server_health():
                        restart_server()
                        current_batch_count = 0  # إعادة ضبط العداد
                
                # محاولة الحصول على شرح الحديث
                response = requests.get(f"{base_url}{hadith_id}", timeout=15)
                data = response.json()
                
                if data and 'data' in data and data['data']:
                    all_explanations.append({
                        'hadith_id': hadith_id,
                        'explanation': data
                    })
                    successful_ids.append(hadith_id)
                    found_count += 1
                    current_batch_count += 1
                else:
                    failed_count += 1
                
                # تحديث البار
                update_progress_bar(pbar, found_count, skipped_count, failed_count, hadith_id)
                
                # حفظ آخر ID تم معالجته
                save_last_id(hadith_id)
                
                # حفظ البيانات كل 10 أحاديث لتقليل عمليات الكتابة المتكررة
                if found_count % 10 == 0:
                    save_data(all_explanations, successful_ids)
                
                # إضافة تأخير عشوائي لتجنب إرهاق السيرفر
                time.sleep(random.uniform(0.3, 0.7))
                
                # التحقق من عدد الأحاديث في الدفعة الحالية
                if current_batch_count >= batch_size:
                    # تحديث رسالة شريط التقدم بدلاً من الطباعة
                    pbar.set_description(f"🔄 إعادة تشغيل السيرفر ({len(all_explanations)} حديث)")
                    
                    # حفظ البيانات قبل إعادة تشغيل السيرفر
                    save_data(all_explanations, successful_ids)
                    
                    restart_server()
                    current_batch_count = 0  # إعادة ضبط العداد
                    
                    # إعادة النص الأصلي للبار
                    pbar.set_description("معالجة الأحاديث")
                
                # تحديث شريط التقدم
                pbar.update(1)
                
            except Exception as e:
                failed_count += 1
                update_progress_bar(pbar, found_count, skipped_count, failed_count, hadith_id)
                
                # محاولة إعادة تشغيل السيرفر في حالة الخطأ
                try:
                    restart_server()
                    current_batch_count = 0  # إعادة ضبط العداد
                except:
                    # وضع تأخير أطول في حالة فشل إعادة التشغيل
                    time.sleep(60)
                
                # تحديث شريط التقدم
                pbar.update(1)
                
                # حفظ آخر حالة على أي حال
                save_last_id(hadith_id)
                save_data(all_explanations, successful_ids)
    
    # حفظ البيانات النهائية
    save_data(all_explanations, successful_ids)
    
    print(f"\n🔹 إجمالي الشروحات التي تم العثور عليها: {found_count}")
    print(f"🔹 إجمالي الأحاديث التي تم تخطيها: {skipped_count}")
    print(f"🔹 إجمالي الأحاديث التي فشل استرجاعها: {failed_count}")
    print("✅ تم حفظ البيانات في ملف 'all_hadiths_explanations2.json'")
    print("✅ تم حفظ المعرفات الناجحة في ملف 'successful_hadith_ids2.json'")

In [8]:
# تشغيل البرنامج
if __name__ == "__main__":
    try:
        crawl_hadith_explanations()
    except KeyboardInterrupt:
        print("\n\n⚠️ تم إيقاف البرنامج يدويًا.")
        print("✓ تم بالفعل حفظ البيانات بشكل تلقائي.")
        print(f"✅ آخر معرف حديث تم معالجته: {load_last_id()}")
        print(f"✅ عدد الشروحات التي تم حفظها: {len(load_existing_data())}")

بدء العملية من معرف الحديث: 866
عدد الشروحات المحفوظة مسبقًا: 0
عدد الأحاديث الناجحة المسجلة: 0


معالجة الأحاديث:   0%|          | 13/87565 [00:11<21:35:01,  1.13حديث/s, عثر=12, تخطى=0, فشل=2, آخر_معالجة=879]



⚠️ تم إيقاف البرنامج يدويًا.
✓ تم بالفعل حفظ البيانات بشكل تلقائي.
✅ آخر معرف حديث تم معالجته: 879
✅ عدد الشروحات التي تم حفظها: 10


#220000

In [ ]:
import subprocess
import time

def shutdown_server():
    print("\n🛑 Shutting down server...")
    try:
        # إيقاف السيرفر
        subprocess.run(["taskkill", "/F", "/IM", "node.exe"], check=True)
        time.sleep(2)  # انتظار للتأكد من إيقاف السيرفر
        print("✅ Server shut down successfully")
    except Exception as e:
        print(f"❌ Error shutting down server: {str(e)}")
        raise

# استدعاء الدالة لإيقاف السيرفر
shutdown_server()

In [1]:
import json
import os
from tqdm import tqdm
import requests
import time
import random
import subprocess

def restart_server():
    """إعادة تشغيل السيرفر بدون طباعة رسائل كثيرة"""
    try:
        # إيقاف جميع عمليات Node.js بطريقة آمنة
        try:
            if os.name == 'nt':  # Windows
                subprocess.run(["taskkill", "/F", "/IM", "node.exe"], 
                               shell=True, capture_output=True, timeout=30)
            else:  # Linux/Mac
                subprocess.run(["pkill", "-f", "node"], 
                               shell=True, capture_output=True, timeout=30)
        except:
            pass
        
        # انتظار للتأكد من إغلاق جميع العمليات
        time.sleep(3)
        
        # تغيير المسار إلى مجلد السيرفر
        server_dir = "D:/repo/Markting/dorar-hadith-api"
        original_dir = os.getcwd()  # حفظ المسار الحالي للعودة إليه لاحقًا
        
        if not os.path.exists(server_dir):
            raise Exception(f"مجلد السيرفر غير موجود: {server_dir}")
        
        os.chdir(server_dir)
        
        # تشغيل السيرفر في الخلفية بدون نافذة عائمة
        if os.name == 'nt':  # Windows
            # استخدام subprocess.CREATE_NO_WINDOW لتشغيل بدون نوافذ عائمة
            startupinfo = subprocess.STARTUPINFO()
            startupinfo.dwFlags |= subprocess.STARTF_USESHOWWINDOW
            startupinfo.wShowWindow = 0  # SW_HIDE
            
            # استخدام CMD مع /c لإخفاء النافذة
            process = subprocess.Popen(
                "cmd /c npm start",
                shell=True,
                startupinfo=startupinfo,
                stdout=subprocess.PIPE,
                stderr=subprocess.PIPE
            )
        else:  # Linux/Mac
            process = subprocess.Popen(
                "npm start > server.log 2>&1 &",
                shell=True
            )
        
        # العودة إلى المسار الأصلي
        os.chdir(original_dir)
        
        # انتظار حتى يبدأ السيرفر مع محاولات متعددة
        max_retries = 12
        retry_count = 0
        server_started = False
        
        while retry_count < max_retries and not server_started:
            try:
                time.sleep(5)  
                response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
                if response.status_code == 200:
                    server_started = True
                    break
            except requests.RequestException:
                retry_count += 1
        
        if not server_started:
            # محاولة بديلة لبدء السيرفر
            try:
                os.chdir(server_dir)
                
                if os.name == 'nt':  # Windows
                    startupinfo = subprocess.STARTUPINFO()
                    startupinfo.dwFlags |= subprocess.STARTF_USESHOWWINDOW
                    startupinfo.wShowWindow = 0  # SW_HIDE
                    
                    subprocess.Popen(
                        "cmd /c npm start",
                        shell=True,
                        startupinfo=startupinfo,
                        stdout=subprocess.PIPE,
                        stderr=subprocess.PIPE
                    )
                else:  # Linux/Mac
                    os.system("npm start > server.log 2>&1 &")
                
                os.chdir(original_dir)
                
                time.sleep(10)
                
                response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
                if response.status_code == 200:
                    server_started = True
            except:
                pass
                
        if not server_started:
            raise Exception("فشل في بدء تشغيل السيرفر بعد كل المحاولات")
            
    except Exception as e:
        raise

def test_server_health():
    """اختبار حالة السيرفر"""
    try:
        response = requests.get("http://localhost:5000/v1/site/sharh/1", timeout=10)
        return response.status_code == 200
    except:
        return False

def recover_lost_hadiths():
    """
    استعادة الأحاديث المفقودة من الملفات الاحتياطية
    """
    # 1. تحميل البيانات الحالية (المتبقية)
    try:
        with open('all_hadiths_explanations2.json', 'r', encoding='utf-8') as f:
            current_data = json.load(f)
            print(f"تم تحميل {len(current_data)} حديث من الملف الحالي")
    except FileNotFoundError:
        current_data = []
        print("لم يتم العثور على ملف البيانات الحالي، سيتم إنشاء ملف جديد")
    
    # 2. تحميل قائمة معرفات الأحاديث الناجحة
    try:
        with open('successful_hadith_ids2.json', 'r', encoding='utf-8') as f:
            successful_ids = json.load(f)
            print(f"تم تحميل {len(successful_ids)} معرف حديث ناجح")
    except FileNotFoundError:
        successful_ids = []
        print("لم يتم العثور على ملف المعرفات الناجحة")
    
    # 3. فحص معرفات الأحاديث الموجودة حالياً
    current_ids = set(item['hadith_id'] for item in current_data)
    print(f"عدد الأحاديث الموجودة حالياً: {len(current_ids)}")
    
    # 4. تحديد المعرفات المفقودة
    missing_ids = set(successful_ids) - current_ids
    missing_ids_list = sorted(list(missing_ids))  # تحويل إلى قائمة مرتبة
    print(f"عدد الأحاديث المفقودة: {len(missing_ids_list)}")
    
    if not missing_ids_list:
        print("لا توجد أحاديث مفقودة للاستعادة!")
        return
    
    # 5. استعادة البيانات من API المحلي
    base_url = "http://localhost:5000/v1/site/sharh/"
    recovered_count = 0
    failed_count = 0
    batch_count = 0  # عداد للدفعة الحالية
    batch_size = 70  # حجم الدفعة (إعادة تشغيل السيرفر بعد كل 70 حديث)
    
    # إنشاء مجلد للنسخ الاحتياطية إذا لم يكن موجوداً
    backup_dir = "hadith_backups"
    os.makedirs(backup_dir, exist_ok=True)
    
    # عمل نسخة احتياطية من البيانات الحالية قبل التعديل
    timestamp = time.strftime("%Y%m%d_%H%M%S")
    backup_file = os.path.join(backup_dir, f"backup_before_recovery_{timestamp}.json")
    with open(backup_file, 'w', encoding='utf-8') as f:
        json.dump(current_data, f, ensure_ascii=False, indent=4)
    print(f"تم حفظ نسخة احتياطية من البيانات الحالية في: {backup_file}")
    
    # التأكد من أن السيرفر يعمل قبل البدء
    if not test_server_health():
        print("❌ السيرفر غير متاح، جاري إعادة تشغيله قبل البدء...")
        restart_server()
        
    print("جاري استعادة الأحاديث المفقودة...")
    
    # استعادة الأحاديث المفقودة
    with tqdm(total=len(missing_ids_list), desc="استعادة الأحاديث", unit="حديث") as pbar:
        for hadith_id in missing_ids_list:
            try:
                # فحص حالة السيرفر بشكل عشوائي
                if random.random() < 0.05:  # 5% احتمالية للاختبار
                    if not test_server_health():
                        print("\n⚠️ السيرفر غير متاح، جاري إعادة تشغيله...")
                        restart_server()
                        batch_count = 0  # إعادة ضبط العداد
                
                # محاولة استرجاع الحديث من الخادم المحلي
                response = requests.get(f"{base_url}{hadith_id}", timeout=15)
                
                if response.status_code == 200:
                    data = response.json()
                    
                    if data and 'data' in data and data['data']:
                        # إضافة الحديث إلى البيانات الحالية
                        current_data.append({
                            'hadith_id': hadith_id,
                            'explanation': data
                        })
                        recovered_count += 1
                        batch_count += 1
                    else:
                        failed_count += 1
                else:
                    failed_count += 1
                
                # تحديث شريط التقدم
                pbar.set_postfix({
                    "تم استعادته": recovered_count,
                    "فشل": failed_count,
                    "الدفعة": batch_count
                })
                pbar.update(1)
                
                # إضافة تأخير عشوائي لتجنب إرهاق السيرفر
                time.sleep(random.uniform(0.3, 0.7))
                
                # حفظ البيانات كل 10 أحاديث
                if recovered_count % 70 == 0:
                    with open('all_hadiths_explanations2.json', 'w', encoding='utf-8') as f:
                        json.dump(current_data, f, ensure_ascii=False, indent=4)
                
                # إعادة تشغيل السيرفر بعد كل 70 حديث تمت استعادته
                if batch_count >= batch_size:
                    pbar.set_description(f"🔄 إعادة تشغيل السيرفر ({recovered_count} حديث)")
                    
                    # حفظ البيانات قبل إعادة تشغيل السيرفر
                    with open('all_hadiths_explanations2.json', 'w', encoding='utf-8') as f:
                        json.dump(current_data, f, ensure_ascii=False, indent=4)
                    
                    restart_server()
                    batch_count = 0  # إعادة ضبط العداد
                    
                    # إعادة النص الأصلي للبار
                    pbar.set_description("استعادة الأحاديث")
                
            except Exception as e:
                print(f"\nخطأ في استعادة الحديث {hadith_id}: {str(e)}")
                failed_count += 1
                
                # محاولة إعادة تشغيل السيرفر في حالة الخطأ
                try:
                    print("⚠️ جاري محاولة إعادة تشغيل السيرفر بعد حدوث خطأ...")
                    restart_server()
                    batch_count = 0  # إعادة ضبط العداد
                except:
                    # وضع تأخير أطول في حالة فشل إعادة التشغيل
                    print("❌ فشل في إعادة تشغيل السيرفر، انتظار 60 ثانية...")
                    time.sleep(60)
                
                # حفظ البيانات بعد الخطأ
                with open('all_hadiths_explanations2.json', 'w', encoding='utf-8') as f:
                    json.dump(current_data, f, ensure_ascii=False, indent=4)
                
                pbar.update(1)
    
    # حفظ البيانات النهائية
    with open('all_hadiths_explanations2.json', 'w', encoding='utf-8') as f:
        json.dump(current_data, f, ensure_ascii=False, indent=4)
    
    print(f"\n✅ تمت استعادة {recovered_count} حديث من أصل {len(missing_ids_list)}")
    print(f"❌ فشل استعادة {failed_count} حديث")
    print(f"📊 إجمالي عدد الأحاديث الحالي: {len(current_data)}")
    print("✅ تم حفظ البيانات في ملف 'all_hadiths_explanations2.json'")

# استدعاء الدالة لاستعادة البيانات
if __name__ == "__main__":
    try:
        recover_lost_hadiths()
    except KeyboardInterrupt:
        print("\n\n⚠️ تم إيقاف البرنامج يدويًا.")
        print("✓ سيتم محاولة حفظ البيانات الحالية...")
        # تحميل البيانات الحالية لطباعة الإحصائيات
        try:
            with open('all_hadiths_explanations2.json', 'r', encoding='utf-8') as f:
                current_data = json.load(f)
            print(f"✅ عدد الأحاديث التي تم حفظها: {len(current_data)}")
        except:
            print("❌ فشل في تحميل البيانات الحالية")

تم تحميل 13949 حديث من الملف الحالي
تم تحميل 26105 معرف حديث ناجح
عدد الأحاديث الموجودة حالياً: 13944
عدد الأحاديث المفقودة: 16606
تم حفظ نسخة احتياطية من البيانات الحالية في: hadith_backups\backup_before_recovery_20250418_104245.json
❌ السيرفر غير متاح، جاري إعادة تشغيله قبل البدء...
جاري استعادة الأحاديث المفقودة...


استعادة الأحاديث:  52%|█████▏    | 8615/16606 [2:39:39<4:05:45,  1.85s/حديث, تم استعادته=8615, فشل=0, الدفعة=5]                   


خطأ في استعادة الحديث 33218: HTTPConnectionPool(host='localhost', port=5000): Read timed out. (read timeout=15)
⚠️ جاري محاولة إعادة تشغيل السيرفر بعد حدوث خطأ...


استعادة الأحاديث:  62%|██████▏   | 10371/16606 [3:15:19<3:12:14,  1.85s/حديث, تم استعادته=10370, فشل=1, الدفعة=5]                    


خطأ في استعادة الحديث 35046: HTTPConnectionPool(host='localhost', port=5000): Read timed out. (read timeout=15)
⚠️ جاري محاولة إعادة تشغيل السيرفر بعد حدوث خطأ...


استعادة الأحاديث: 100%|██████████| 16606/16606 [5:17:03<00:00,  1.15s/حديث, تم استعادته=16604, فشل=2, الدفعة=4]                      



✅ تمت استعادة 16604 حديث من أصل 16606
❌ فشل استعادة 2 حديث
📊 إجمالي عدد الأحاديث الحالي: 30553
✅ تم حفظ البيانات في ملف 'all_hadiths_explanations2.json'
